In [2]:
!pip install lxml html5lib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import glob
import os
import warnings
import re

# 경고 메시지 무시
warnings.filterwarnings("ignore")

# ==========================================
# 1. 파일 경로 설정
# ==========================================
target_dir = r"C:\Users\1108j\OneDrive\바탕 화면\data prep\맛집 추출_시군구\Gwanak_Data"
file_list = glob.glob(os.path.join(target_dir, "*.xls*"))

print(f"📂 폴더: {target_dir}")
print(f"📄 파일 개수: {len(file_list)}개")

all_data = []

# ==========================================
# 2. 만능 파일 읽기 함수 (HTML/Excel 자동 판별)
# ==========================================
def read_weird_excel(filepath):
    try:
        # 1. 그냥 엑셀로 읽기 (혹시 윗줄에 제목이 있으면 header=1로 조정)
        return pd.read_excel(filepath)
    except:
        try:
            # 2. HTML로 읽기 (euc-kr)
            dfs = pd.read_html(filepath, encoding='euc-kr')
            # 표가 여러 개일 수 있으니 가장 데이터 많은 걸 선택하거나 첫 번째 걸 선택
            if len(dfs) >= 1: return dfs[0]
            else: return None
        except:
            try:
                # 3. HTML로 읽기 (utf-8)
                dfs = pd.read_html(filepath, encoding='utf-8')
                return dfs[0]
            except:
                return None

# ==========================================
# 3. 데이터 로딩 (실시간 중계)
# ==========================================
print("\n🚀 데이터 로딩 시작!")

for i, file_path in enumerate(file_list):
    file_name = os.path.basename(file_path)
    print(f"[{i+1}/{len(file_list)}] 읽는 중... {file_name}", end='\r', flush=True)
    
    df = read_weird_excel(file_path)
    
    if df is not None:
        # [중요] 혹시 첫 줄이 '업무추진비 목록' 같은 제목이라면 삭제하는 안전장치
        if '집행장소' not in df.columns:
            # 첫 번째 행을 헤더로 다시 설정 시도
            new_header = df.iloc[0] 
            df = df[1:] 
            df.columns = new_header

        all_data.append(df)

print(f"\n✅ 로딩 완료! 분석을 시작합니다.")

# ==========================================
# 4. 데이터 병합 및 분석 (수정된 이름표 적용!)
# ==========================================
if not all_data:
    print("\n❌ 데이터를 못 읽었습니다.")
else:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # 컬럼 이름 공백 제거 (매우 중요)
    combined_df.columns = combined_df.columns.astype(str).str.strip()
    
    print(f"📊 병합 완료 ({len(combined_df)}행).")

    # ---------------------------------------------------------------
    # [수정됨] 사진 보고 똑같이 맞춘 이름표!
    # ---------------------------------------------------------------
    col_place = '집행장소'
    col_day = '집행일'
    col_time = '집행시간'
    col_amount = '집행금액(원)'  # 사진에 (원) 있음
    col_people = '대상인원수'    # 사진에 (명) 없음! (여기가 문제였음)

    # 필수 컬럼 체크
    if col_place not in combined_df.columns:
        print(f"🚨 [에러] '{col_place}' 컬럼이 없습니다. 현재 컬럼: {list(combined_df.columns)}")
    else:
        # 1. 장소명 정제 (괄호 제거)
        combined_df['정제된장소명'] = combined_df[col_place].astype(str).str.split('(').str[0].str.strip()
        
        # 2. 숫자 변환 (콤마 제거)
        for col in [col_amount, col_people]:
            if col in combined_df.columns:
                # 숫자가 아닌 것들은 0으로 변환 (errors='coerce')
                combined_df[col] = pd.to_numeric(combined_df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
            else:
                print(f"⚠️ 경고: '{col}' 컬럼을 찾을 수 없습니다.")

        # 3. 1인당 금액 계산
        if col_people in combined_df.columns and col_amount in combined_df.columns:
             combined_df['1인당금액'] = combined_df.apply(
                lambda x: x[col_amount] / x[col_people] if x[col_people] > 0 else 0, axis=1
            )
        else:
            print("⚠️ '금액' 또는 '인원' 정보가 부족하여 [가성비 분석]은 건너뜁니다.")
            combined_df['1인당금액'] = 0 # 에러 방지용 0 채우기

        # 4. 날짜 변환
        if col_day in combined_df.columns:
            combined_df[col_day] = pd.to_datetime(combined_df[col_day], errors='coerce')

        # 5. 시간 변환 (시:분:초 or 시:분) -> '시'(Hour) 추출
        def extract_hour(time_val):
            try:
                time_str = str(time_val).strip()
                if ':' in time_str: # 12:30 형태
                    return int(time_str.split(':')[0])
                elif '시' in time_str: # 12시 형태
                    return int(re.search(r'(\d+)시', time_str).group(1))
                else:
                    return None
            except:
                return None

        if col_time in combined_df.columns:
            combined_df['시각'] = combined_df[col_time].apply(extract_hour)
        else:
            combined_df['시각'] = None

        # ----------------------------------------------------
        # 랭킹 산출
        # ----------------------------------------------------
        def get_ranking(df, title):
            if df.empty: return pd.DataFrame(columns=['식당이름', '방문횟수'])
            ranking = df['정제된장소명'].value_counts().reset_index()
            ranking.columns = ['식당이름', '방문횟수']
            
            # 평균 1인당 금액 정보 추가
            if '1인당금액' in df.columns:
                avg_price = df.groupby('정제된장소명')['1인당금액'].mean().round(0)
                ranking = ranking.merge(avg_price, left_on='식당이름', right_index=True, how='left')
                ranking.rename(columns={'1인당금액': '평균1인당금액'}, inplace=True)
            return ranking

        print("🧠 랭킹 계산 중...", flush=True)
        
        # 1. 전체 랭킹
        rank_total = get_ranking(combined_df, "전체")
        
        # 2. 최신순
        if col_day in combined_df.columns:
            df_recent = combined_df.sort_values(by=col_day, ascending=False)
        else:
            df_recent = combined_df

        # 3. 점심/저녁
        rank_lunch = get_ranking(combined_df[(combined_df['시각'] >= 11) & (combined_df['시각'] <= 14)], "점심")
        rank_dinner = get_ranking(combined_df[combined_df['시각'] >= 17], "저녁")
        
        # 4. 가성비/고급
        rank_cheap = get_ranking(combined_df[(combined_df['1인당금액'] >= 10000) & (combined_df['1인당금액'] < 20000)], "1만원대")
        rank_expensive = get_ranking(combined_df[combined_df['1인당금액'] >= 30000], "3만원이상")

        # 엑셀 저장
        save_path = os.path.join(target_dir, "관악구_맛집_결과_최종.xlsx")
        print("💾 저장 중...", flush=True)
        
        with pd.ExcelWriter(save_path) as writer:
            rank_total.head(100).to_excel(writer, sheet_name='1_전체랭킹', index=False)
            df_recent.head(100).to_excel(writer, sheet_name='2_최신데이터', index=False)
            rank_lunch.head(50).to_excel(writer, sheet_name='3_점심', index=False)
            rank_dinner.head(50).to_excel(writer, sheet_name='4_저녁', index=False)
            
            if not rank_cheap.empty:
                rank_cheap.head(50).to_excel(writer, sheet_name='5_1만원대', index=False)
            if not rank_expensive.empty:
                rank_expensive.head(50).to_excel(writer, sheet_name='6_3만원이상', index=False)

        print(f"\n🎉 [성공] 분석 끝! 아래 파일을 열어보세요:\n📂 {save_path}")

📂 폴더: C:\Users\1108j\OneDrive\바탕 화면\data prep\맛집 추출_시군구\Gwanak_Data
📄 파일 개수: 38개

🚀 데이터 로딩 시작!
[38/38] 읽는 중... 업무추진비_20260129 (9).xlss
✅ 로딩 완료! 분석을 시작합니다.
📊 병합 완료 (56170행).
🧠 랭킹 계산 중...
💾 저장 중...

🎉 [성공] 분석 끝! 아래 파일을 열어보세요:
📂 C:\Users\1108j\OneDrive\바탕 화면\data prep\맛집 추출_시군구\Gwanak_Data\관악구_맛집_결과_최종.xlsx
